In [1]:
# %matplotlib widget
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import numpy as np
import torch
import cv2
import scipy.signal
import scipy.ndimage
import sys
import os
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
sys.path.append("../../")
import helpers
import glm


%load_ext autoreload
%autoreload 2

In [5]:
def moving_window_array(array: torch.Tensor, window_size, overlap) -> torch.Tensor:
    shape = array.shape
    strides = (
        shape[-1] * (window_size - overlap),
        (window_size - overlap),
        shape[-1],
        1
    )
    shape = (
        int((shape[-2] - window_size) / (window_size - overlap)) + 1,
        int((shape[-1] - window_size) / (window_size - overlap)) + 1,
        window_size,
        window_size,
    )
    return torch.as_strided(
        array, size=shape, stride=strides)

In [6]:
def index_min(lst):
    index = lst.argmin()
    col = lst.shape[1]
    return index//col, index % col  # row, col

In [7]:
testing_0 = helpers.imread_normalized_float_grayscale(os.path.join("test", "0.jpg"))
testing_0 = np.where(testing_0 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testing_1 = helpers.imread_normalized_float_grayscale(os.path.join("test", "1.jpg"))
testing_1 = np.where(testing_1 > 0.5, 1.0, 0.0)  # TEMPORARY, TO MAKE TEST MORE IDEAL
testtest_0 = np.array(np.split(np.arange(64), 8))
testtest_1 = np.array(np.split(np.arange(64), 8)) - 1

In [8]:
test0 = helpers.imread_normalized_float_grayscale(os.path.join("plume simulation", "00101.jpg"))
test1 = helpers.imread_normalized_float_grayscale(os.path.join("plume simulation", "00102.jpg"))

In [9]:
# Assumptions for simple version: ignoring restrictions and bug checks, assuming window size fits into img with no remainder, overlap is strictly less than window size, images are numpy arrays and perfect squares
def calculate(img1, img2, window_size, overlap):
    # split img1 into windows, 3d array, with all possible overlaps:
    img1_windowed = moving_window_array(torch.from_numpy(img1), window_size, overlap)
    img2_windowed = moving_window_array(torch.from_numpy(img2), window_size, overlap)
    # Check correlation from 1 image to next one, for each window, to get movement of each patch.
    # Searching whole image for correlation for each window is redundant, only searching for two windows across. Assumption: Each image split into at least a 2x2 grid of windows.
    offset = 4  # for now
    window_dim = int(img2_windowed.shape[0])
    vector_field_shape = list((*img2_windowed.shape[:2], 4))
    vector_field = np.zeros(vector_field_shape)
    for row_1, window_1_row in enumerate(img1_windowed):
            for col_1, window_1 in enumerate(window_1_row):
                row_min = max((row_1-offset), 0)
                row_max = min((row_1+offset) + 1, window_dim)
                col_min = max((col_1-offset), 0)
                col_max = min((col_1+offset) + 1, window_dim)
                rel_windows_2 = img2_windowed[row_min:row_max, col_min:col_max, :, : ]
                rel_diffs = np.zeros(rel_windows_2.shape[:2])
                for row_2, rel_row_2 in enumerate(rel_windows_2):
                    for col_2, rel_wind_2 in enumerate(rel_row_2):
                        rel_diffs[row_2,col_2] = torch.sum(torch.abs(window_1 - rel_wind_2)).item() / (rel_windows_2.shape[0] * rel_windows_2.shape[1])
                smallest_rel_index = index_min(rel_diffs)
                wind_2_abs_loc = np.array((row_min, col_min)) + smallest_rel_index
                vect = wind_2_abs_loc - (row_1, col_1)
                vect_loc = (row_1 * (window_size - overlap) + window_size // 2, (col_1 * (window_size - overlap) + window_size // 2))
                vector_field[row_1, col_1, :] = np.array((*vect, *vect_loc))
    return np.array(vector_field)
bc = calculate(testing_0, testing_1, 32, 31)


In [10]:
def plot_velocity(img, field):
    vx, vy, x, y =  np.reshape(field, (field.shape[0] * field.shape[1], 4)).T
        
            # new_x = x/piv_gen._scale
            # new_y = y/piv_gen._scale
    plt.imshow(img, cmap='gray')
    plt.quiver(x, y, vx, vy, color='red')
    plt.show()

In [11]:
plot_velocity(testing_0, bc)

In [ ]:
#### OLD VERSION ###
# img2_4d = img2_windowed.reshape(window_dim, window_dim, img2_windowed.shape[-2], img2_windowed.shape[-1])
#     vector_field_shape = list((*img2_4d.shape[:2], 2))
#     vector_field = np.zeros(vector_field_shape)
#     for i, window_1 in enumerate(img1_windowed):
#         row = i // window_dim
#         col = i % window_dim
#         rel_windows_2 = img2_4d[max((row-4),0):min((row+4) + 1,window_dim), max((col-4),0):min((col+4) + 1,window_dim), :, : ]
#         rel_diffs = np.zeros(rel_windows_2.shape[:2])
#         for j, rel_cols_2 in enumerate(rel_windows_2):
#             for k, rel_wind_2 in enumerate(rel_cols_2):
#                 rel_diffs[j,k] = torch.sum(torch.abs(window_1 - rel_wind_2)).item() / (window_size ** 2)
#         
#         vector_field[row, col, :] = index_min(rel_diffs)
#     return np.array(vector_field)
# bc = calculate(testing_0, testing_1, 64, 61)
# WINDOWING FUNCTIONS
# assuming for now that window size fits perfectly into img, and img is 2d np array perfect square  
# def windowed_old(img, window_size):
#     return np.lib.stride_tricks.as_strided(img, shape=(int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size), strides =(img.strides[0] * window_size, img.strides[1] * window_size, img.strides[0], img.strides[1]) , writeable=False)

# TODO: UNUSED CODE, IGNORE THIS CELL
# # Just to visualize images
# fig, ax = plt.subplots()
# ax.axis("off")
# ax.set_title("Projected points")
# ax.imshow(windowed(testing_0, 8).reshape((128,128)), cmap="gray")
# # ax.imshow(testing_0, cmap="gray")
# 
# fig.tight_layout()
# plt.show()
# print(testing_0)

#     # return img.reshape((int(img.shape[0] / window_size), int(img.shape[1] / window_size), window_size, window_size))
#     
# def windowed(img, window_size):
#     return ressample(img, window_size)
# def ressample(arr, N):
#     A = []
#     for v in np.vsplit(arr, arr.shape[0] // N):
#         A.extend([*np.hsplit(v, arr.shape[0] // N)])
#     return np.array(A)
# 
# 
        # up_offs_row = max(row - window_size // (window_size - overlap) - 1, 0)
        # down_offs_row = min(row + window_size // (window_size - overlap) + 1, window_dim)
        # left_offs_col = max(col - window_size // (window_size - overlap) - 1, 0)
        # right_offs_col = min(col + window_size // (window_size - overlap) + 1, window_dim)
        # current_wind_2 = img2_4d[left_offs_col:right_offs_col, up_offs_row:down_offs_row, :, :]
        # testtesttorch = torch.from_numpy(testtest)
# print(testtesttorch)
# print(testtesttorch.shape)
# print(moving_window_array(testtesttorch, 4, 3).shape)
